In [23]:
import pandas as pd

In [24]:
tweets = pd.read_csv("train.csv")
tweets.set_index("id",inplace=True)
tweets

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...
10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1


In [25]:
# Encuentra hashtags en el tweet y los reemplaza por las palabras que hay en el separandolas por espacios

# tweet = "Buen día #ATomarCafe"
# limpiarHashtags(tweet) = buen día a tomar cafe

def limpiarHashtags(tweet):

    #print(tweet)
    purged = str(tweet)
    
    index_start = tweet.find("#")
    while not index_start < 0:

        index_end = purged.find(" ",index_start)
        if(index_end < 0): hashtag = purged[index_start:]
        else: hashtag = purged[index_start:index_end]
        
        purged = purged.replace(hashtag,extractHashtag(hashtag))
        #print("Hashtag_index:",index_start, "end_hashtag:",index_end, "hashtag:",hashtag+".")
        index_start = purged.find("#")
        #print("Purged:",purged,"Next hashtag:",index_start)

        
        
    return purged

def extractHashtag(hashtag):
    
    hashtag = hashtag.lstrip("#")
    extracted = ""
    for i in range(len(hashtag)):
        if 'A' <= hashtag[i] <= 'Z':
            extracted += " "
        extracted += hashtag[i]
        
    return extracted.lower()

In [26]:
# Considerando que los links que hay son abreviados y no aportan ninguna informacion más que "existe un link externo"
# se encuentran los links y se eliminan

def limpiarLinks(tweet):
    
    purged = str(tweet)
    
    index_start = tweet.find("http")
    while not index_start < 0:
        
        index_end = purged.find(" ",index_start)
        if(index_end < 0): link = purged[index_start:]
        else: link = purged[index_start:index_end]
        
        purged = purged.replace(link,"")
        index_start = purged.find("http")
    
    return purged

In [27]:
# Los tags (etiquetas) tampoco aportan algna informacion, solo un nombre ficticio.
# Se encuentran los tags y se eliminan

def limpiarTags(tweet):
    
    purged = str(tweet)
    
    index_start = tweet.find("@")
    while not index_start < 0:
        
        index_end = purged.find(" ",index_start)
        if(index_end < 0): tag = purged[index_start:]
        else: tag = purged[index_start:index_end]
        
        purged = purged.replace(tag,"")
        index_start = purged.find("@")
    
    return purged
        

In [28]:
# Se queda solo con caracteres alfabeticos (a-z) 

def limpiarCaracteresInvalidos(string):
    
    purged = string.lower()
    for c in string.lower():
        if not 'a' <= c or not c <= 'z':
            if not c == ' ':
                purged = purged.replace(c,'')
            
    return purged

In [29]:
tweets["purged"] = tweets["text"].agg(limpiarLinks).agg(limpiarTags).agg(limpiarHashtags).agg(limpiarCaracteresInvalidos)
tweets

,keyword,location,text,target,purged
id,,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in...
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as ...
...,...,...,...,...,...
10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,two giant cranes holding a bridge collapse int...
10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,the out of control wild fires in california ...
10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,m utckm s of volcano hawaii


In [ ]:
words = pd.DataFrame(columns= ["tweet_id","word","context","target"])

for tweet in tweets.reset_index().values:

    for w in tweet[5].split(" "):
        #print("Size:",words.size,"/100..")
        words = words.append(pd.DataFrame([[tweet[0],w,tweet[5],tweet[4]]],columns= ["tweet_id","word","context","target"]))
        if(len(words) % 1500 == 0): print(words.sample(1))

words

In [ ]:
words.to_csv("words.csv")